In [1]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from colormath.color_objects import sRGBColor, XYZColor
from colormath.color_conversions import convert_color
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
MF = np.genfromtxt("CMF_5nm.csv", delimiter=',') 
print(MF.shape)
CMF = MF[:, 1:4]

(89, 4)


# Delta and Gap Colors

We convert XYZ to RGB and display

In [3]:
# display delta colors
fig = plt.figure()
ax = fig.gca(projection='3d')
for i, O in enumerate(CMF):
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    ax.scatter(i, -5, s=400, color=rgb) 
for i, O in enumerate(CMF):
    C = XYZColor(1-O[0], 1-O[1], 1-O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    ax.scatter(i, 5, s=400, color=rgb) 
plt.show()

# delta and gap colors on the color solid

Plot the color solid and graph delta and gap colors

In [4]:
def bump(I, x0, xf):
    out = np.zeros(len(I))
    if x0 < xf:
        out[x0:xf] = 1
    elif x0 > xf:
        out[x0:] = 1
        out[0:xf] = 1
    return out

def Optimal(wl1, wl2, cmf, normalized=False):
    """
    Optimal windowing function with wrapping
    """
    X =  np.dot(cmf[:,0], bump(cmf, wl1, wl2))
    Y =  np.dot(cmf[:,1], bump(cmf, wl1, wl2))
    Z =  np.dot(cmf[:,2], bump(cmf, wl1, wl2))
    XYZ = [X, Y, Z]
    if normalized:
        Norm = sum(CMF)
        return (XYZ[0]/Norm[0], XYZ[1]/Norm[1], XYZ[2]/Norm[2])
    else:
        return (XYZ[0], XYZ[1], XYZ[2])

In [5]:
C0 = []
C1 = []
C2 = []
RGB = []
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlim(sum(CMF[:,0]))
ax.set_ylim(sum(CMF[:,1]))
ax.set_zlim(sum(CMF[:,2]))
L = CMF.shape[0]
for i in range(0, L, 1):
    for j in range(0, L, 1):
        O = Optimal(i, (i+j)%L, CMF)
        C0.append( O[0] )
        C1.append( O[1] )
        C2.append( O[2] )
        C = XYZColor(O[0]/sum(CMF)[0], O[1]/sum(CMF)[1], O[2]/sum(CMF)[2])
        crgb = convert_color(C, sRGBColor)
        r = crgb.rgb_r
        g = crgb.rgb_g
        b = crgb.rgb_b
        r = r if r > 0 else 0
        g = g if g > 0 else 0
        b = b if b > 0 else 0
        r = r if r < 1 else 1
        g = g if g < 1 else 1
        b = b if b < 1 else 1
        rgb = [r, g, b]
        RGB.append(rgb)
ax.scatter(C0, C1, C2, c=RGB)

# line info
xvec = np.linspace(0, sum(CMF[:,0]), 100)

# plot delta colors and lines
C0 = []
C1 = []
C2 = []
RGB = []
for O in CMF:
    C = XYZColor(O[0], O[1], O[2])
    crgb = convert_color(C, sRGBColor)
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    RGB.append(rgb)
    yvec = (O[1]/O[0])*(xvec-O[0]) + O[1]
    zvec = (O[2]/O[0])*(xvec-O[0]) + O[2]
    ax.plot(xvec, yvec, zvec, color=rgb)
    
ax.scatter(C0, C1, C2, s=400, c=RGB) 
# plot gap colors and lines
C0 = []
C1 = []
C2 = []
RGB = []
for O in CMF:
    C = XYZColor(1-O[0], 1-O[1], 1-O[2])
    crgb = convert_color(C, sRGBColor)
    r = crgb.rgb_r
    g = crgb.rgb_g
    b = crgb.rgb_b
    r = r if r > 0 else 0
    g = g if g > 0 else 0
    b = b if b > 0 else 0
    r = r if r < 1 else 1
    g = g if g < 1 else 1
    b = b if b < 1 else 1
    rgb = [r, g, b]
    RGB.append(rgb)
    [w1, w2, w3] = sum(CMF)
    yvec = (O[1]/O[0])*(xvec-w1) + w2
    zvec = (O[2]/O[0])*(xvec-w1) + w3
    ax.plot(xvec, yvec, zvec, color=rgb)
    O = sum(CMF)-O
    C0.append( O[0] )
    C1.append( O[1] )
    C2.append( O[2] )
    
ax.scatter(C0, C1, C2, s=400, c=RGB) 
plt.show()